# get nips paper links 

In [135]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

def parse_proceding_links(base_url, attrs={}):
    # Send a GET request to retrieve the HTML content
    response = requests.get(base_url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all <a> tags with the title and href attributes
    paper_links = soup.find_all("a", attrs={"href": True})

    # Initialize empty lists to store the titles and URLs
    titles = []
    urls = []

    # Extract the title and URL from each <a> tag and store them in the respective lists
    for link in paper_links:
        title = link.get_text(strip=True)
        url = base_url + link["href"]
        if 'paper_files' not in url:
            continue
        titles.append(title)
        urls.append(url)

    # Create a pandas DataFrame from the collected data
    df = pd.DataFrame({"Title": titles, "URL": urls})
    return df




def parse_paper_links(url, attrs={}):
    # Send a GET request to retrieve the HTML content
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all <a> tags with the title and href attributes
    paper_links = soup.find_all("a", attrs={"title": True, "href": True})

    # Initialize empty lists to store the titles and URLs
    titles = []
    urls = []
    authors = []
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    base_url

    # Extract the title and URL from each <a> tag and store them in the respective lists
    for link in paper_links:
        title = link.get_text(strip=True)
        url = base_url + link["href"]
        author = link.find_next_sibling("i").get_text(strip=True)

        titles.append(title)
        urls.append(url)
        authors.append(author)

    # Create a pandas DataFrame from the collected data
    df = pd.DataFrame({"title": titles, "URL": urls, "authors": authors})
    return df


def get_metadata():
    proceedings = parse_proceding_links('https://papers.nips.cc/')
    proceedings['year'] = proceedings.URL.apply(lambda x:x.split('/')[-1])
    dfs = []
    for _,r in tqdm(proceedings.iterrows()):
        df = parse_paper_links(r.URL)
        df['year'] = r.year
        dfs.append(df)
    papers_metadata = pd.concat(dfs)
    return papers_metadata

# get more detailed data 


def paper_detailed_metadata(url):
    # Send a GET request to retrieve the HTML content
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the article title
    title = soup.find("h4").text.strip()

    # Extract the authors
    authors = soup.find("h4", string="Authors").find_next("p").text.strip()

    # Extract the abstract
    abstract = soup.find("h4", string="Abstract").find_next("p").text.strip()

    # Extract the paper link
    paper_link = soup.find("a", string="Paper")["href"]

    # Extract the supplementary link
    supplementary_link = soup.find("a", string="Supplemental")["href"]

    # Extract the bibtex link
    bibtex_link = soup.find("a", string="Bibtex")["href"]

    # Return the extracted information as a dictionary
    data = {
        "title": title,
        "author": authors,
        "abstract": abstract,
        "link": paper_link,
        "supp_link": supplementary_link,
        "bib_link": bibtex_link,
    }
    return data

# Example usage 
# papers_meta = []
# for url in tqdm(paper_links.iloc[:10].URL.unique()):
#     try:
#         papers_meta.append(paper_detailed_metadata(url))
#     except:
#         pass
# papers_meta = pd.DataFrame(papers_meta)
# papers_meta




papers_metadata = get_metadata()
papers_metadata

,title,URL,authors,year
0,Federated Submodel Optimization for Hot and Co...,https://papers.nips.cc/paper_files/paper/2022/...,"Yucheng Ding, Chaoyue Niu, Fan Wu, Shaojie Tan...",2022
1,On Kernelized Multi-Armed Bandits with Constra...,https://papers.nips.cc/paper_files/paper/2022/...,"Xingyu Zhou, Bo Ji",2022
2,Geometric Order Learning for Rank Estimation,https://papers.nips.cc/paper_files/paper/2022/...,"Seon-Ho Lee, Nyeong Ho Shin, Chang-Su Kim",2022
3,Structured Recognition for Generative Models w...,https://papers.nips.cc/paper_files/paper/2022/...,"Changmin Yu, Hugo Soulat, Neil Burgess, Manees...",2022
4,NAS-Bench-Graph: Benchmarking Graph Neural Arc...,https://papers.nips.cc/paper_files/paper/2022/...,"Yijian Qin, Ziwei Zhang, Xin Wang, Zeyang Zhan...",2022
...,...,...,...,...
85,Connecting to the Past,https://papers.nips.cc/paper_files/paper/1987/...,Bruce MacDonald,1987
86,PARTITIONING OF SENSORY DATA BY A CORTICAL NET...,https://papers.nips.cc/paper_files/paper/1987/...,"Richard Granger, Jose Ambros-Ingerson, Howard ...",1987
87,A Dynamical Approach to Temporal Pattern Proce...,https://papers.nips.cc/paper_files/paper/1987/...,"W. Stornetta, Tad Hogg, Bernardo Huberman",1987
88,Minkowski-r Back-Propagation: Learning in Conn...,https://papers.nips.cc/paper_files/paper/1987/...,"Stephen Hanson, David Burr",1987


In [ ]:
import requests
import feedparser
import math
import requests
import random
import time
import os


# Function to search arXiv for a paper
def search_arxiv(main_paper, start = 0, max_results = 10):
    # filter items
    paper_dict = main_paper.to_dict()
    paper_dict = {k:v for k,v in paper_dict.items() if str(v)!='nan'}

    # Define the base URL and query parameters
    base_url = "http://export.arxiv.org/api/query"
    title = paper_dict.get('title', '')
    authors = paper_dict.get('authors', '')
    year = paper_dict.get('year', '')
    abstract = paper_dict.get('abstract', '')

    
    # Construct the search query
    search_query = f"all:{title} all:{authors} all:{year}"

    # Construct the full URL
    full_url = f"{base_url}?search_query={search_query}&start={start}&max_results={max_results}"

    # Make the GET request
    response = requests.get(full_url)

    # Parse the response
    feed = feedparser.parse(response.content)

    papers = []
    for paper in feed.entries:
        # print(paper)
        paper.authors = ', '.join(author.name for author in paper.authors)
        sim = paper_sim(main_paper,paper)
        if sim >= .7:
            papers.append(paper)
    # Return the feed entries (i.e., the papers)
    return papers

def paper_sim(query_paper, paper):
    # Convert titles and authors to lowercase
    title1 = query_paper.title.lower()
    authors1 = query_paper.authors.lower()
    title2 = paper.title.lower()
    authors2 = paper.authors.lower()

    # Calculate Jaccard similarity for titles
    title_set1 = set(title1.split())
    title_set2 = set(title2.split())
    title_similarity = len(title_set1.intersection(title_set2)) / len(title_set1.union(title_set2))

    # Calculate Jaccard similarity for authors
    authors_set1 = set(authors1.split(', '))
    authors_set2 = set(authors2.split(', '))
    authors_similarity = len(authors_set1.intersection(authors_set2)) / len(authors_set1.union(authors_set2))

    # Calculate the overall similarity score as the average of title and authors similarity
    similarity_score = (title_similarity + authors_similarity) / 2

    return similarity_score

# Function to download a file from a paper record
def download_arxiv_source(paper):
    arxiv_id = paper.id.split('/')[-1]
    url = f"https://arxiv.org/e-print/{arxiv_id}"
    local_filename = f"./sources/{arxiv_id}.tar.gz"
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    if local_filename:
        print(f"Downloaded {local_filename}")
    return local_filename

bar = tqdm(papers_metadata.iloc[:20].iterrows(),total=len(papers_metadata))
for _,main_paper in bar:
    bar.set_postfix(title=main_paper.title)
    papers = search_arxiv(main_paper, max_results=4)

    for paper in papers:
        download_arxiv_source(paper);


  0%|          | 0/16746 [00:00<?, ?it/s]

Downloaded ./sources/2109.07704v4.tar.gz
Downloaded ./sources/2203.15589v1.tar.gz
Downloaded ./sources/2209.05212v2.tar.gz
Downloaded ./sources/2203.09675v3.tar.gz
Downloaded ./sources/2205.11266v2.tar.gz
Downloaded ./sources/2206.08155v2.tar.gz
Downloaded ./sources/2210.08367v1.tar.gz
Downloaded ./sources/2301.00355v2.tar.gz
Downloaded ./sources/2201.13097v2.tar.gz
Downloaded ./sources/2206.10870v1.tar.gz
Downloaded ./sources/2103.16277v1.tar.gz
Downloaded ./sources/2210.07702v1.tar.gz


In [2]:
import os
import tarfile
from tqdm.auto import tqdm

# Specify the directory containing the tar.gz files
directory = './sources/'

# Iterate over all files in the directory
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.gz'):
        try:
            filepath = os.path.join(directory, filename)

            # Create a directory name without the '.tar' extension
            output_dir = os.path.splitext(filepath)[0]
            output_dir = output_dir.replace('.tar', '')
            output_dir = output_dir.replace('.gz', '')

            os.makedirs(output_dir, exist_ok=True)

            # Extract the tar.gz file into the output directory
            with tarfile.open(filepath, 'r:gz') as tar:
                tar.extractall(output_dir)
        except:
            pass

  0%|          | 0/408 [00:00<?, ?it/s]